In [1]:
import os
import numpy as np
import random
import pickle
import copy
from matplotlib import pyplot as plt

# reproducibility
np.random.seed(100)
random.seed(100)



In [2]:
# This isn't gonna run without reworking the path...
## So let's take care of that here:
import sys
from pathlib import Path
# assumes this notebook lives one sub-directory down from your repo root
repo_root = Path().resolve().parents[0]   # “parent” folder
sys.path.insert(0, str(repo_root))
print("inserting onto sys.path:", repo_root)
print("exists?", repo_root.exists())
print("contents:", list(repo_root.iterdir()))

from experiment_params import *
from cost_funcs import *
from fl_sim_client import *
from fl_sim_server import *
from shared_globals import *
from utils import *  #load_model_logs

inserting onto sys.path: C:\Users\kdmen\Repos\personalization-privacy-risk\Main_PythonVersion\Main_Final
exists? True
contents: [WindowsPath('C:/Users/kdmen/Repos/personalization-privacy-risk/Main_PythonVersion/Main_Final/cost_funcs.py'), WindowsPath('C:/Users/kdmen/Repos/personalization-privacy-risk/Main_PythonVersion/Main_Final/experiment_params.py'), WindowsPath('C:/Users/kdmen/Repos/personalization-privacy-risk/Main_PythonVersion/Main_Final/fl_sim_base.py'), WindowsPath('C:/Users/kdmen/Repos/personalization-privacy-risk/Main_PythonVersion/Main_Final/fl_sim_client.py'), WindowsPath('C:/Users/kdmen/Repos/personalization-privacy-risk/Main_PythonVersion/Main_Final/fl_sim_server.py'), WindowsPath('C:/Users/kdmen/Repos/personalization-privacy-risk/Main_PythonVersion/Main_Final/forked'), WindowsPath('C:/Users/kdmen/Repos/personalization-privacy-risk/Main_PythonVersion/Main_Final/ipynbs'), WindowsPath('C:/Users/kdmen/Repos/personalization-privacy-risk/Main_PythonVersion/Main_Final/kcs_ml_i

In [3]:
# GLOBALS
GLOBAL_METHOD       = "NOFL"   # or "PFAFO", "NOFL", "FEDAVG"
OPT_METHOD          = 'FULLSCIPYMIN' if GLOBAL_METHOD=="NOFL" else 'GD'
GLOBAL_ROUNDS       = NOFL_NUM_ROUNDS if GLOBAL_METHOD=="NOFL" else NUM_GLOBAL_ROUNDS
LOCAL_ROUND_THRESHOLD = 1 if GLOBAL_METHOD=="NOFL" else LRT
NUM_STEPS           = 1 if GLOBAL_METHOD=="NOFL" else NUM_FL_STEPS

# load your data
with open(path + SAVED_DATASET, 'rb') as fp:
    cond_training_and_labels_lst = pickle.load(fp)


In [4]:
base_path = 'C:\\Users\\kdmen\\Repos\\personalization-privacy-risk\\Main_PythonVersion\\Main_Final\\results\\IEEE_Final'


In [5]:
def load_model_logs(cv_results_path, filename, num_folds=7):
    extraction_dict = dict()
    for i in range(num_folds):
        h5_path = os.path.join(cv_results_path, filename+f"{i}.h5")
        with h5py.File(h5_path, 'r') as f:
            a_group_key = list(f.keys())
            for key in a_group_key:
                if key=="client_local_model_log":
                    client_keys = list(f[key])
                    for ck in client_keys:
                        ed_key = f"{ck}_fold{i}"  # Does this never update from or something...
                        if len(list(f[key][ck]))==0:
                            pass
                        else:
                            extraction_dict[ed_key] = list(f[key][ck])
                elif key=="global_dec_log" and "NOFL" not in filename:
                    ed_key = f"{key}_fold{i}"
                    extraction_dict[ed_key] = list(f[key])
                else:
                    pass
    return extraction_dict

In [6]:
# Load in the model logs

# CROSS
#cpfa_model_dict = load_model_logs(base_path+r'\\Cross_PFAFO', 'GD_PFAFO_KFold')
#cfa_model_dict = load_model_logs(base_path+r'\\Cross_FEDAVG', 'GD_FEDAVG_KFold')
cnofl_model_dict = load_model_logs(base_path+r'\\Cross_NOFL', 'FULLSCIPYMIN_NOFL_KFold')
# INTRA
#ipfa_model_dict = load_model_logs(base_path+r'\\Intra_PFAFO', 'GD_PFAFO_KFold')
#ifa_model_dict = load_model_logs(base_path+r'\\Intra_FEDAVG', 'GD_FEDAVG_KFold')
#inofl_model_dict = load_model_logs(base_path+r'\\Intra_NOFL', 'FULLSCIPYMIN_NOFL_KFold')

In [7]:
cnofl_model_dict.keys()

dict_keys(['S0_client_local_model_log_fold0', 'S10_client_local_model_log_fold0', 'S11_client_local_model_log_fold0', 'S12_client_local_model_log_fold0', 'S13_client_local_model_log_fold0', 'S1_client_local_model_log_fold0', 'S2_client_local_model_log_fold0', 'S3_client_local_model_log_fold0', 'S4_client_local_model_log_fold0', 'S5_client_local_model_log_fold0', 'S6_client_local_model_log_fold0', 'S7_client_local_model_log_fold0', 'S8_client_local_model_log_fold0', 'S9_client_local_model_log_fold0', 'S0_client_local_model_log_fold1', 'S10_client_local_model_log_fold1', 'S11_client_local_model_log_fold1', 'S12_client_local_model_log_fold1', 'S13_client_local_model_log_fold1', 'S1_client_local_model_log_fold1', 'S2_client_local_model_log_fold1', 'S3_client_local_model_log_fold1', 'S4_client_local_model_log_fold1', 'S5_client_local_model_log_fold1', 'S6_client_local_model_log_fold1', 'S7_client_local_model_log_fold1', 'S8_client_local_model_log_fold1', 'S9_client_local_model_log_fold1', '

In [8]:
import re
from collections import defaultdict

def extract_models_by_fold(model_dict):
    """
    Groups model snapshots by fold and orders each list by subject number.

    Parameters:
    - model_dict: dict mapping keys like 'S0_client_local_model_log_fold0' to arrays

    Returns:
    - A dict: { 'fold0': [array_S0_f0, array_S1_f0, ...],
                'fold1': [...],
                ... }
    """
    # Temporary storage: fold_index → list of (subject_index, array)
    temp = defaultdict(list)
    pattern = re.compile(r'^S(\d+)_client_local_model_log_fold(\d+)$')

    for key, arr in model_dict.items():
        m = pattern.match(key)
        if not m:
            continue
        subj_idx = int(m.group(1))
        fold_idx = int(m.group(2))
        temp[fold_idx].append((subj_idx, arr))

    # Build the final ordered dict
    ordered_by_fold = {}
    for fold_idx in sorted(temp.keys()):
        # Sort by subject index, then extract arrays in order
        sorted_items = sorted(temp[fold_idx], key=lambda x: x[0])
        ordered_by_fold[f'fold{fold_idx}'] = [arr for _, arr in sorted_items]

    return ordered_by_fold



In [ ]:
def report_model_distances(models_by_fold):
    """
    For each subject, computes and prints:
      - Standard deviation of the L2 norms of that subject’s final models across folds
      - Euclidean distances between fold0 and fold1..foldN for that subject

    Parameters:
    - models_by_fold: dict mapping 'fold0','fold1',... to lists of weight‐vectors (lists or arrays)
    """
    # Sort fold keys numerically
    fold_keys = sorted(models_by_fold.keys(), key=lambda k: int(k.replace('fold','')))
    
    # Determine number of subjects from the first fold
    first_fold = models_by_fold[fold_keys[0]]
    num_subjects = len(first_fold)
    
    for subj_idx in range(num_subjects):
        # Gather this subject’s final model from each fold (if present)
        vecs = []
        present_folds = []
        for fk in fold_keys:
            fold_list = models_by_fold[fk]
            if subj_idx < len(fold_list):
                vecs.append(np.asarray(fold_list[subj_idx]))  # to array
                present_folds.append(fk)
        
        if not vecs:
            print(f"Subject S{subj_idx}: no models found in any fold.")
            continue
        
        # Compute L2 norms and their std
        norms = [np.linalg.norm(v) for v in vecs]
        std_norm = float(np.std(norms))
        print(f"Subject S{subj_idx} model norm std: {std_norm:.4f}")
        
        # Compute distances only between fold0 and fold1..foldN
        base_vec = vecs[0]
        base_fold = present_folds[0]
        for j in range(1, len(vecs)):
            d = float(np.linalg.norm(base_vec - vecs[j]))
            f2 = present_folds[j]
            print(f"  Distance S{subj_idx} {base_fold} → {f2}: {d:.4f}")
        print()


In [10]:
models_by_fold = extract_models_by_fold(cnofl_model_dict)


In [11]:
report_model_distances(models_by_fold)

Subject S0 model norm std: 30.8827
  Distance S0 fold0 → fold1: 97.4963
  Distance S0 fold0 → fold2: 0.0000
  Distance S0 fold0 → fold3: 0.0000
  Distance S0 fold0 → fold4: 0.0000
  Distance S0 fold0 → fold5: 0.0000
  Distance S0 fold0 → fold6: 0.0000

Subject S1 model norm std: 17.7940
  Distance S1 fold0 → fold1: 0.0000
  Distance S1 fold0 → fold2: 0.0000
  Distance S1 fold0 → fold3: 0.0000
  Distance S1 fold0 → fold4: 61.6453
  Distance S1 fold0 → fold5: 0.0000
  Distance S1 fold0 → fold6: 0.0000

Subject S2 model norm std: 30.8230
  Distance S2 fold0 → fold1: 0.0000
  Distance S2 fold0 → fold2: 0.0000
  Distance S2 fold0 → fold3: 0.0000
  Distance S2 fold0 → fold4: 0.0000
  Distance S2 fold0 → fold5: 0.0000
  Distance S2 fold0 → fold6: 96.7841

Subject S3 model norm std: 6.6928
  Distance S3 fold0 → fold1: 0.0000
  Distance S3 fold0 → fold2: 0.0000
  Distance S3 fold0 → fold3: 0.0000
  Distance S3 fold0 → fold4: 32.5216
  Distance S3 fold0 → fold5: 0.0000
  Distance S3 fold0 → fold

> The above trend can be explained by the fact that Local+Cross, even though we change which users are in the testing set, we never actually change what the training datasets. Thus, combined with the fact that Local is deterministic when seeded, we get the same model every time (we are training on the same data and find the same global minimum). The one decoder that is not equal to all the others is when taht subject is used as the test subject, and the subject has the init decoder that has never been trained.

In [12]:
fold0_list = models_by_fold['fold0']  # list of arrays in S0, S1, S2, ... order

for idx, ele in enumerate(fold0_list):
    print(f"S{idx}, len {len(ele)}")

S0, len 11
S1, len 11
S2, len 11
S3, len 11
S4, len 11
S5, len 11
S6, len 11
S7, len 1
S8, len 11
S9, len 11
S10, len 1
S11, len 11
S12, len 11
S13, len 11


In [13]:
full_client_lst = []
final_model_log = []
for i in range(NUM_USERS):
    print(f"i: {i}, len: {len(fold0_list[i])}")
    final_model = fold0_list[i][-1]
    final_model_log.append(copy.deepcopy(final_model))
    full_client_lst.append(Client(
                        i,
                        copy.deepcopy(final_model),
                        OPT_METHOD,
                        cond_training_and_labels_lst[i],
                        DATA_STREAM,
                        lr=LR,
                        smoothbatch_lr=SMOOTHBATCH_LR, 
                        beta=BETA,
                        scenario="INTRA",
                        local_round_threshold=LOCAL_ROUND_THRESHOLD,
                        starting_update=STARTING_UPDATE,
                        global_method=GLOBAL_METHOD,
                        num_steps=NUM_STEPS,
                        test_split_type="TEST_ON_PRE"
                        )
                    )


i: 0, len: 11
i: 1, len: 11
i: 2, len: 11
i: 3, len: 11
i: 4, len: 11
i: 5, len: 11
i: 6, len: 11
i: 7, len: 1
i: 8, len: 11
i: 9, len: 11
i: 10, len: 1
i: 11, len: 11
i: 12, len: 11
i: 13, len: 11


In [14]:
loss_log = []
vel_err_log = []
dec_err_log = []
for i, cli in enumerate(full_client_lst):
    if i in [7, 10]:
        print(f"Skipping fold0 untrained withheld test subject {i}")
    else:
        loss, (vel_err, dec_err) = cli.test_metrics(final_model_log[i], which='LOCAL', return_cost_func_comps=True)
        loss_log.append(loss)
        vel_err_log.append(vel_err)
        dec_err_log.append(dec_err)


Skipping fold0 untrained withheld test subject 7
Skipping fold0 untrained withheld test subject 10


In [15]:
loss_log

[0.033731959614955315,
 0.17869039280435778,
 0.005489657242389587,
 0.0041713756040382185,
 0.0037994851538065474,
 0.027784306071114744,
 0.01191656494307179,
 0.09165999849553777,
 0.022914542322215126,
 0.6734361897012279,
 0.013154315070118907,
 0.01289734988917947]

In [16]:
np.mean(loss_log)

0.08997051140933443

In [17]:
np.std(loss_log)

0.18250343428180282

In [18]:
np.mean(vel_err_log)

0.08993263292856124

In [19]:
np.mean(dec_err_log)

3.787848077317579e-05

In [20]:
old_intra_err_log = [0.017689582, 
                    0.02488066, 
                    0.877951066, 
                    0.014488665, 
                    0.016317067, 
                    0.071500494, 
                    0.005124895, 
                    0.001997069, 
                    0.006421436, 
                    0.017807219, 
                    0.007818029, 
                    0.147865809, 
                    0.014568378, 
                    0.013714958]

print(f"Len intra_log: {len(old_intra_err_log)}")
print(np.mean(old_intra_err_log))

Len intra_log: 14
0.08843895192857143


In [21]:
cross_err_log = [0.263164298, #S0
                0.235297268,  #S1
                0.556111058,  #S10
                0.948417739,  #S11
                0.070724384,  #S12
                0.248437074,  #S13
                0.010915889,  #S2
                0.070971476,  #S3
                0.26031072,   #S4
                0.048793324,  #S5
                0.296195863,  #S6
                0.029562334,  #S7
                0.403155799,  #S8
                0.138793677]  #S9

print(f"Len cross_log: {len(cross_err_log)}")
print(np.mean(cross_err_log))

Len cross_log: 14
0.2557750645


> Shouldn't cross only be 12 elements? We should not be testing the 2 untrained withheld subjects (from the testing fold)... The above data is already averaged across the folds

In [22]:

def average_client_loss_across_folds(
    models_by_fold,
    cond_training_and_labels_lst,
    num_folds,
    num_users
):
    """
    For each fold 0..num_folds-1:
      - Retrieves each subject's model snapshots from models_by_fold['fold{fold_idx}']
      - Skips any subject with <=1 snapshots (withheld clients)
      - Instantiates a Client with the final snapshot and evaluates test_metrics
      - Accumulates the loss per subject across folds
    Returns a DataFrame with each Subject's average loss across the folds they appeared in.
    """
    # Initialize storage for losses
    loss_by_client = {i: [] for i in range(num_users)}

    for fold_idx in range(num_folds):
        fold_key = f'fold{fold_idx}'
        fold_list = models_by_fold[fold_key]
        print(f"\nProcessing {fold_key}, subjects found: {len(fold_list)}")

        for i, snapshots in enumerate(fold_list):
            # Skip withheld clients (<=1 snapshot)
            if len(snapshots) <= 1:
                print(f"  Skipping S{i} in {fold_key} (only {len(snapshots)} snapshot(s))")
                continue

            # Use the last snapshot as the final model
            final_model = snapshots[-1]

            # Instantiate the Client
            client = Client(
                i,
                copy.deepcopy(final_model),
                OPT_METHOD,
                cond_training_and_labels_lst[i],
                DATA_STREAM,
                lr=LR,
                smoothbatch_lr=SMOOTHBATCH_LR,
                beta=BETA,
                scenario="INTRA",
                local_round_threshold=LOCAL_ROUND_THRESHOLD,
                starting_update=STARTING_UPDATE,
                global_method=GLOBAL_METHOD,
                num_steps=NUM_STEPS,
                test_split_type="TEST_ON_PRE"
            )

            # Evaluate and record loss
            loss, _ = client.test_metrics(final_model, which='LOCAL', return_cost_func_comps=True)
            loss_by_client[i].append(loss)
            print(f"    Fold {fold_idx}, Subject S{i} → loss {loss:.4f}")

    # Build a DataFrame of average loss per subject
    records = []
    for i in range(num_users):
        losses = loss_by_client[i]
        avg_loss = np.mean(losses) if losses else np.nan
        records.append({"Subject": f"S{i}", "Avg_Loss": avg_loss})

    return pd.DataFrame(records)


In [23]:
df = average_client_loss_across_folds(
    models_by_fold, cond_training_and_labels_lst,
    num_folds=7,
    num_users=14)


Processing fold0, subjects found: 14
    Fold 0, Subject S0 → loss 0.0337
    Fold 0, Subject S1 → loss 0.1787
    Fold 0, Subject S2 → loss 0.0055
    Fold 0, Subject S3 → loss 0.0042
    Fold 0, Subject S4 → loss 0.0038
    Fold 0, Subject S5 → loss 0.0278
    Fold 0, Subject S6 → loss 0.0119
  Skipping S7 in fold0 (only 1 snapshot(s))
    Fold 0, Subject S8 → loss 0.0917
    Fold 0, Subject S9 → loss 0.0229
  Skipping S10 in fold0 (only 1 snapshot(s))
    Fold 0, Subject S11 → loss 0.6734
    Fold 0, Subject S12 → loss 0.0132
    Fold 0, Subject S13 → loss 0.0129

Processing fold1, subjects found: 14
  Skipping S0 in fold1 (only 1 snapshot(s))
    Fold 1, Subject S1 → loss 0.1787
    Fold 1, Subject S2 → loss 0.0055
    Fold 1, Subject S3 → loss 0.0042
  Skipping S4 in fold1 (only 1 snapshot(s))
    Fold 1, Subject S5 → loss 0.0278
    Fold 1, Subject S6 → loss 0.0119
    Fold 1, Subject S7 → loss 0.0550
    Fold 1, Subject S8 → loss 0.0917
    Fold 1, Subject S9 → loss 0.0229
    

In [ ]:
print(df.shape)
df.head()

(14, 2)


,Subject,Avg_Loss
0,S0,0.033732
1,S1,0.178690
2,S2,0.005490
3,S3,0.004171
4,S4,0.003799


In [27]:
df["Avg_Loss"]

0     0.033732
1     0.178690
2     0.005490
3     0.004171
4     0.003799
5     0.027784
6     0.011917
7     0.054979
8     0.091660
9     0.022915
10    0.014028
11    0.673436
12    0.013154
13    0.012897
Name: Avg_Loss, dtype: float64